# LMKR RAG Chatbot - Jupyter Notebook

This notebook implements a production-ready RAG (Retrieval-Augmented Generation) chatbot for LMKR using LangChain, HuggingFace LLM, and a custom vector database.



## Cell 1: Project Setup & Dependencies Installation

Install all required packages for LangChain, HuggingFace, FAISS, and utilities.

In [1]:


!pip install -U langchain langchain-community langchain-huggingface langchain-text-splitters
!pip install -U huggingface-hub transformers torch
!pip install sentence-transformers hf_xet
!pip install faiss-cpu
!pip install python-dotenv
!pip install pypdf requests
!pip install accelerate bitsandbytes


  Using cached huggingface_hub-1.1.5-py3-none-any.whl.metadata (13 kB)


**What this cell does:**
- Installs LangChain and integration libraries
- Installs HuggingFace models and utilities
- Sets up FAISS (Facebook AI Similarity Search) for vector database
- Installs data loading utilities (PDF parsing, web scraping)

## Cell 2: Imports & Configuration

In [2]:
# ---- Standard Libraries ----
import os
from datetime import datetime

# ---- HuggingFace (LangChain Integration) ----
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import (
    HuggingFaceEmbeddings,
    HuggingFacePipeline
)

# ---- LangChain Core ----
from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter


# ---- Document Loaders & Vectorstores (Community packages) ----
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.vectorstores import FAISS

# ---- RAG / Retrieval (no RetrievalQA in v1.x, use LCEL) ----
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# ---- LLMs ----
from langchain_community.llms import HuggingFaceHub  # if you use HF hub models
# or:
# from langchain_community.chat_models import ChatOpenAI, ChatAnthropic, etc.

# ---- Utility ----
import torch
import json
from typing import Dict, List, Dict




**What this cell does:**
- Imports all necessary libraries from LangChain, HuggingFace, and transformers
- Sets up logging for debugging and monitoring
- Loads environment variables (useful for API keys or configurations)
- Prepares utilities for text splitting, embeddings, and vector storage

## Cell 3: Configuration Parameters

Centralized configuration for the entire RAG pipeline. Tune these to optimize chatbot performance.

In [3]:
DATA_SOURCE_TYPE = "text"  # Options: "text", "csv", "pdf", "url"
DATA_PATH = "./Data/lmkr_data/lmkr_combined.txt"  # Path to your LMKR data file

CHUNK_SIZE = 1000
CHUNK_OVERLAP = 500

# 3. EMBEDDING MODEL CONFIG
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
MODEL_KWARGS = {"device": "cuda" if torch.cuda.is_available() else "cpu"}
ENCODE_KWARGS = {"normalize_embeddings": False}

# 4. VECTOR DATABASE CONFIG
VECTOR_DB_PATH = "./vector_db/faiss_lmkr"
USE_EXISTING_DB = False

# 5. LLM MODEL CONFIG
LLM_MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"
LLM_MAX_LENGTH = 100
LLM_TEMPERATURE = 0.1
LLM_TOP_P = 0.9



RETRIEVER_K = 1
CHAIN_TYPE = "stuff"

# 7. PROJECT METADATA
PROJECT_NAME = "LMKR-RAG-Chatbot"
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
RESULTS_DIR = f"./results/{TIMESTAMP}"

print(f"✓ Configuration loaded: {PROJECT_NAME}")
print(f"✓ Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print(f"✓ Results directory: {RESULTS_DIR}")

✓ Configuration loaded: LMKR-RAG-Chatbot
✓ Device: Tesla T4
✓ Results directory: ./results/20251125_120712


**What this cell does:**
- Centralizes all hyperparameters and configurations
- Explains trade-offs for each parameter
- Makes it easy to experiment with different models and settings
- Provides alternative model suggestions


## Cell 4: Data Loading & Exploration

Load your LMKR data from various sources (text, CSV, PDF, web).

In [4]:
# ============================================================
# LOAD LMKR DATA
# Simple and clean - just raw text, no URLs
# ============================================================

try:
    with open(DATA_PATH, 'r', encoding='utf-8') as f:
        raw_data = f.read()

    print(f"✓ Successfully loaded LMKR data")
    print(f"  - Total characters: {len(raw_data):,}")
    print(f"  - Estimated tokens: {len(raw_data) // 4:,}")
    print(f"  - Ready for RAG pipeline")

except FileNotFoundError:
    print(f"❌ File not found: {DATA_PATH}")
    print(f"Please ensure lmkr_combined.txt exists in ./data/lmkr_data/")
    raw_data = ""
except Exception as e:
    print(f"❌ Error loading file: {e}")
    raw_data = ""


✓ Successfully loaded LMKR data
  - Total characters: 5,903
  - Estimated tokens: 1,475
  - Ready for RAG pipeline


**What this cell does:**
- Loads LMKR data from various sources (text, CSV, PDF)
- Handles different data formats automatically
- Provides basic data statistics
- Shows preview to verify data loaded correctly


## Cell 5: Data Preprocessing & Text Chunking

Clean, normalize, and split text into manageable chunks for embedding.

In [5]:
# ============================================================
# PREPROCESS DATA & CREATE CHUNKS
# This prepares raw text for embedding and retrieval
# ============================================================

def preprocess_text(text: str) -> str:
    """
    Clean and preprocess text.
    """
    import re
    text = re.sub(r'\\n+', '\\n', text)
    text = text.strip()
    return text

def chunk_text(text: str, chunk_size: int, overlap: int) -> List[str]:
    """
    Split text into chunks using RecursiveCharacterTextSplitter.
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap,
        separators=["\\n\\n", "\\n", " ", ""]
    )
    chunks = splitter.split_text(text)
    return chunks

# Execute preprocessing
print("Preprocessing text...")
cleaned_data = preprocess_text(raw_data)

print("Chunking text...")
text_chunks = chunk_text(cleaned_data, CHUNK_SIZE, CHUNK_OVERLAP)

print(f"\\n✓ Chunking Statistics:")
print(f"  - Total chunks: {len(text_chunks)}")
if text_chunks:
    print(f"  - Average chunk size: {sum(len(c) for c in text_chunks) // len(text_chunks)} chars")
    print(f"  - Max chunk size: {max(len(c) for c in text_chunks)} chars")
    print(f"  - Min chunk size: {min(len(c) for c in text_chunks)} chars")
    print(f"\\nFirst chunk preview:\\n{text_chunks[0][:300]}...")

Preprocessing text...
Chunking text...
\n✓ Chunking Statistics:
  - Total chunks: 11
  - Average chunk size: 986 chars
  - Max chunk size: 998 chars
  - Min chunk size: 912 chars
\nFirst chunk preview:\n# LMKR Company Data Collection & Storage Guide

## 📊 COMPREHENSIVE LMKR COMPANY INFORMATION

### **Company Overview**
- **Name:** LMKR
- **Founded:** 1994
- **Headquarters:** Houston, Texas
- **Type:** Privately Held Technology Company
- **Employees:** 201-500
- **Industry:** Oil & Gas, Geoscience, ...


**What this cell does:**
- Cleans and normalizes text
- Splits text into manageable chunks (default 1000 chars)
- Maintains overlap for context continuity
- Provides statistics on chunking effectiveness


## Cell 6: Embedding Model Setup

Initialize embedding model to convert text into vector representations.

In [6]:
# ============================================================
# INITIALIZE EMBEDDING MODEL
# This converts text into vector representations
# ============================================================

print("Loading embedding model...")
print(f"Model: {EMBEDDING_MODEL_NAME}")
print(f"Device: {MODEL_KWARGS['device']}")

embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    model_kwargs=MODEL_KWARGS,
    encode_kwargs=ENCODE_KWARGS
)

# Test embedding on a sample
test_text = "LMKR is a test query"
test_embedding = embeddings.embed_query(test_text)
print(f"\\n✓ Embedding test successful!")
print(f"  - Sample query embedding shape: {len(test_embedding)}")
print(f"  - First 5 values: {test_embedding[:5]}")

Loading embedding model...
Model: sentence-transformers/all-MiniLM-L6-v2
Device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


\n✓ Embedding test successful!
  - Sample query embedding shape: 384
  - First 5 values: [0.052791938185691833, 0.0005524340667761862, -0.05132013186812401, 0.07101751118898392, 0.03979623317718506]


**What this cell does:**
- Loads embedding model from HuggingFace
- Initializes embeddings for your GPU/CPU
- Tests embedding functionality
- Explains model trade-offs

## Cell 7: Vector Database Creation & Indexing

Convert text chunks to embeddings and store in FAISS for fast retrieval.

In [7]:
# ============================================================
# CREATE & INDEX VECTOR DATABASE
# Converts text chunks to embeddings and stores in FAISS
# ============================================================

def create_vector_db(documents: List[str], embeddings, db_path: str):
    """
    Create FAISS vector database from documents.
    """
    print("Creating vector database...")
    print(f"Number of documents to embed: {len(documents)}")

    vector_db = FAISS.from_texts(
        texts=documents,
        embedding=embeddings,
        metadatas=[{"source": f"chunk_{i}"} for i in range(len(documents))]
    )

    os.makedirs(db_path, exist_ok=True)
    vector_db.save_local(db_path)
    print(f"✓ Vector database created and saved to {db_path}")
    return vector_db

def load_vector_db(embeddings, db_path: str):
    """
    Load existing FAISS vector database.
    """
    print(f"Loading vector database from {db_path}...")
    vector_db = FAISS.load_local(db_path, embeddings)
    print(f"✓ Vector database loaded successfully")
    return vector_db

if USE_EXISTING_DB and os.path.exists(VECTOR_DB_PATH):
    vector_store = load_vector_db(embeddings, VECTOR_DB_PATH)
else:
    vector_store = create_vector_db(text_chunks, embeddings, VECTOR_DB_PATH)

print("\\n--- Testing Retriever ---")
test_query = "What is LMKR?"  # TODO: Modify based on domain
retrieved_docs = vector_store.similarity_search(test_query, k=RETRIEVER_K)

print(f"Retrieved {len(retrieved_docs)} documents for query: '{test_query}'")
for i, doc in enumerate(retrieved_docs):
    print(f"\\n[Document {i+1}]")
    print(f"Content: {doc.page_content[:200]}...")

Creating vector database...
Number of documents to embed: 11
✓ Vector database created and saved to ./vector_db/faiss_lmkr
\n--- Testing Retriever ---
Retrieved 1 documents for query: 'What is LMKR?'
\n[Document 1]
Content: # LMKR Company Data Collection & Storage Guide

## 📊 COMPREHENSIVE LMKR COMPANY INFORMATION

### **Company Overview**
- **Name:** LMKR
- **Founded:** 1994
- **Headquarters:** Houston, Texas
- **Type:*...


**What this cell does:**
- Converts all text chunks to embeddings
- Creates FAISS index for fast similarity search
- Saves index to disk for reuse
- Tests retriever with a sample query


## Cell 8: LLM Model Setup (HuggingFace)

Load language model that will generate final responses.

In [8]:
# ============================================================
# LOAD LANGUAGE MODEL (HuggingFace)
# This generates the final responses
# ============================================================

print("Loading LLM from HuggingFace...")
print(f"Model: {LLM_MODEL_NAME}")
print(f"Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_NAME, max_length=LLM_MAX_LENGTH)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id


try:
    model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL_NAME,
        device_map="auto",
        dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    )
    print("✓ Model loaded successfully")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    raise

# Create pipeline
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=LLM_MAX_LENGTH,
    temperature=LLM_TEMPERATURE,
    top_p=LLM_TOP_P,
)

# Wrap in LangChain
llm = HuggingFacePipeline(
    model_id=LLM_MODEL_NAME,
    pipeline=llm_pipeline,
)

print("✓ LLM pipeline created successfully")

# Test LLM
print("\\n--- Testing LLM ---")
test_prompt = "Tell me some of LMKR clients:"
try:
    test_output = llm.invoke(test_prompt)
    print(f"Input: {test_prompt}")
    print(f"Output: {test_output[:200]}")
except Exception as e:
    print(f"Note: LLM test skipped ({type(e).__name__})")
    print(f"LLM is ready to use in RAG chain")

Loading LLM from HuggingFace...
Model: mistralai/Mistral-7B-Instruct-v0.1
Device: Tesla T4


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


✓ Model loaded successfully
✓ LLM pipeline created successfully
\n--- Testing LLM ---
Input: Tell me some of LMKR clients:
Output: Tell me some of LMKR clients:

1. ABB
2. Siemens
3. GE
4. Honeywell
5. Rockwell Automation
6. Schneider Electric
7. Emerson Electric
8. Bosch Rexroth
9. Mitsubishi Electric
10. Ametek
11. National Ins


**What this cell does:**
- Loads tokenizer and model from HuggingFace
- Configures GPU optimization
- Creates text generation pipeline
- Tests LLM functionality

## Cell 9: System Prompt Engineering

Craft system prompts to guide chatbot behavior and response quality.

In [18]:
# ============================================================
# CRAFT SYSTEM PROMPTS FOR CHATBOT
# This is critical for accuracy and response quality
# ============================================================

# Customer service oriented
SYSTEM_PROMPT_TEMPLATE = """Answer this question in your own words based on the provided information. write bla bla before answering.
Context:
{context}

Question: {question}

Answer in 1-2 sentences:"""

ACTIVE_SYSTEM_PROMPT = SYSTEM_PROMPT_TEMPLATE

print("✓ System prompt configured for Mistral-7B")

print("✓ System prompts defined")
print(f"Active prompt: SYSTEM_PROMPT_CUSTOMER_SERVICE")
print(f"\\nPrompt preview:\\n{ACTIVE_SYSTEM_PROMPT[:300]}...")

✓ System prompt configured for Mistral-7B
✓ System prompts defined
Active prompt: SYSTEM_PROMPT_CUSTOMER_SERVICE
\nPrompt preview:\nAnswer this question in your own words based on the provided information. write bla bla before answering.
Context:
{context}

Question: {question}

Answer in 1-2 sentences:...


**What this cell does:**
- Defines multiple system prompt templates
- Explains different prompt strategies
- Provides template for context-aware responses

**Prompt Engineering Tips:**
- Be specific about role and constraints
- Provide clear instructions on using context
- Include examples of desired behavior
- Experiment and measure quality differences

## Cell 10: RAG Chain Assembly

Combine retriever, prompt template, and LLM into complete RAG pipeline.

In [19]:
# ============================================================
# ASSEMBLE RAG CHAIN (MODERN LANGCHAIN APPROACH)
# Combines retriever, prompt template, and LLM
# ============================================================

# Create prompt template
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template=ACTIVE_SYSTEM_PROMPT
)

# Create retriever from vector store
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": RETRIEVER_K}
)

# Helper function to format documents
def format_docs(docs):
    """Format retrieved documents for the prompt"""
    return "\n\n".join(doc.page_content for doc in docs)

# Assemble RAG chain using modern LCEL (LangChain Expression Language)
print("Building RAG chain...")

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

print("✓ RAG chain assembled successfully")
print(f"Chain configuration:")
print(f"  - Retriever K: {RETRIEVER_K}")
print(f"  - LLM: {LLM_MODEL_NAME}")


Building RAG chain...
✓ RAG chain assembled successfully
Chain configuration:
  - Retriever K: 1
  - LLM: mistralai/Mistral-7B-Instruct-v0.1


**What this cell does:**
- Creates prompt template for RAG
- Converts vector store to retriever
- Assembles RAG chain combining all components
- Configures chain parameters

**Chain Types:**
- `stuff`: Concatenate docs (simple, works for small contexts)
- `map_reduce`: Summarize each doc, then combine (for many docs)
- `refine`: Iteratively improve answer (best accuracy, slower)

## Cell 11: Testing & Inference

Test RAG chain with sample queries and evaluate responses.

In [20]:
# ============================================================
# TEST RAG CHATBOT
# Run queries and evaluate responses
# ============================================================

def query_chatbot(rag_chain, retriever, query: str) -> Dict:
    """
    Query the RAG chatbot and return result with sources.
    """
    docs = retriever.invoke(query)
    answer = rag_chain.invoke(query)

    # Extract only the answer portion (remove prompt)
    if "Answer in 1-2 sentences:" in answer:
        answer = answer.split("Answer in 1-2 sentences:")[-1].strip()

    return {
        "query": query,
        "answer": answer,
        "source_documents": docs,
        "timestamp": datetime.now().isoformat()
    }


test_queries = [
    "When was LMKR founded?",
    "What is Gverse?",
    "What are LMKR's ISO-Certified Services?",
    "Name one of LMKR's partnership?",
]

print("=" * 70)
print("TESTING RAG CHATBOT")
print("=" * 70)

test_results = []

for i, query in enumerate(test_queries):
    print(f"\n[Query {i+1}/{len(test_queries)}]")
    print(f"Question: {query}")
    print("-" * 70)

    try:
        result = query_chatbot(rag_chain, retriever, query)  # Pass retriever
        test_results.append(result)

        print(f"Answer: {result['answer']}")


    except Exception as e:
        print(f"Error processing query: {e}")
        import traceback
        traceback.print_exc()


# Save test results
os.makedirs(RESULTS_DIR, exist_ok=True)
results_path = f"{RESULTS_DIR}/test_results.json"

# Convert Document objects to dictionaries for JSON serialization
serializable_results = []
for result in test_results:
    serializable_result = {
        "query": result["query"],
        "answer": result["answer"],
        "source_documents": [
            {
                "content": doc.page_content,
                "metadata": doc.metadata
            }
            for doc in result["source_documents"]
        ],
        "timestamp": result["timestamp"]
    }
    serializable_results.append(serializable_result)

with open(results_path, 'w', encoding='utf-8') as f:
    json.dump(serializable_results, f, indent=2, ensure_ascii=False)

print(f"\n✓ Test results saved to {results_path}")



TESTING RAG CHATBOT

[Query 1/4]
Question: When was LMKR founded?
----------------------------------------------------------------------


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Answer: LMKR was founded in 1994.

[Query 2/4]
Question: What is Gverse?
----------------------------------------------------------------------
Answer: Gverse is a leading-edge geoscience software platform used for exploration and production in the oil and gas industry. It offers a range of features including mapping and geological interpretation, geophysical analysis, petrophysical interpretation, structural modeling, and well and field planning.

[Query 3/4]
Question: What are LMKR's ISO-Certified Services?
----------------------------------------------------------------------
Answer: LMKR offers a range of ISO-certified services, including geoscience interpretation, geophysical analysis, petrophysical interpretation, structural modeling, well and field planning, and real-time seismic attribute analysis.

[Query 4/4]
Question: Name one of LMKR's partnership?
----------------------------------------------------------------------
Answer: LMKR has partnerships with several companies in 

**What this cell does:**
- Tests RAG chain with sample queries
- Captures answers and source documents
- Shows retrieval effectiveness
- Saves results for analysis

**Next Steps After Testing:**
- Evaluate answer quality
- Check if correct documents were retrieved
- Refine prompts if needed
- Adjust retriever K if needed

## Cell 12: Interactive Chat Loop

Enable interactive chat with continuous conversation.

In [ ]:
# ============================================================
# INTERACTIVE CHATBOT
# Allows continuous conversation with the chatbot
# ============================================================

def chat_with_bot(rag_chain, history: List = None):
    """
    Interactive chat interface for the RAG chatbot.
    Type 'quit' to exit.
    """
    if history is None:
        history = []

    print("=" * 70)
    print("LMKR RAG CHATBOT - Interactive Mode")
    print("Type 'quit' to exit, 'clear' to clear history")
    print("=" * 70)

    while True:
        user_input = input("\\nYou: ").strip()

        if user_input.lower() == "quit":
            print("Goodbye!")
            break

        if user_input.lower() == "clear":
            history = []
            print("History cleared")
            continue

        if not user_input:
            continue

        try:
            result = query_chatbot(rag_chain, user_input)
            history.append(result)
            print(f"\\nBot: {result['answer']}")

        except Exception as e:
            print(f"Error: {e}")

    return history

# TODO: Uncomment to run interactive chat
chat_history = chat_with_bot(rag_chain)

LMKR RAG CHATBOT - Interactive Mode
Type 'quit' to exit, 'clear' to clear history
\nYou: hi
Error: query_chatbot() missing 1 required positional argument: 'query'
\nYou: hello
Error: query_chatbot() missing 1 required positional argument: 'query'
\nYou: LMKR
Error: query_chatbot() missing 1 required positional argument: 'query'


**What this cell does:**
- Provides interactive chat interface
- Maintains conversation history
- Shows bot responses in real-time

**Usage:**
- Uncomment the last line to activate
- Type your questions
- Type 'quit' to exit

## Cell 13: Evaluation & Metrics

Measure and track chatbot quality metrics.

In [13]:
# ============================================================
# EVALUATE CHATBOT QUALITY
# Measure accuracy, relevance, and response quality
# ============================================================

def evaluate_responses(test_results: List[Dict]) -> Dict:
    """
    Evaluate chatbot response quality.
    """
    metrics = {
        "total_queries": len(test_results),
        "avg_answer_length": 0,
        "avg_sources_retrieved": 0,
        "queries_with_sources": 0,
    }

    if not test_results:
        return metrics

    # Calculate metrics
    total_length = sum(len(r["answer"]) for r in test_results)
    total_sources = sum(len(r["source_documents"]) for r in test_results)

    metrics["avg_answer_length"] = total_length // len(test_results)
    metrics["avg_sources_retrieved"] = total_sources / len(test_results)
    metrics["queries_with_sources"] = sum(
        1 for r in test_results if len(r["source_documents"]) > 0
    )

    return metrics

# Evaluate test results
if test_results:
    metrics = evaluate_responses(test_results)
    print("\\n" + "=" * 70)
    print("EVALUATION METRICS")
    print("=" * 70)
    for key, value in metrics.items():
        print(f"{key}: {value}")

    # Save metrics
    metrics_path = f"{RESULTS_DIR}/metrics.json"
    with open(metrics_path, 'w') as f:
        json.dump(metrics, f, indent=2)
    print(f"\\n✓ Metrics saved to {metrics_path}")

\n======================================================================
EVALUATION METRICS
total_queries: 4
avg_answer_length: 196
avg_sources_retrieved: 1.0
queries_with_sources: 4
\n✓ Metrics saved to ./results/20251125_120712/metrics.json


**What this cell does:**
- Calculates basic quality metrics
- Provides framework for custom evaluation
- Saves metrics for comparison

**Evaluation Framework:**
- Manual scoring: Rate 1-5 (relevance, accuracy, clarity)
- Automated metrics: BLEU, ROUGE, similarity scores
- Comparison: Test different models/prompts

## Cell 14: Save & Deploy Configuration

Save all settings for reproducibility and future reference.

In [14]:
# ============================================================
# SAVE PROJECT CONFIGURATION
# Store settings for reproducibility
# ============================================================

def save_project_config():
    """
    Save all project configuration to JSON.
    """
    config = {
        "project_name": PROJECT_NAME,
        "timestamp": TIMESTAMP,
        "data": {
            "source_type": DATA_SOURCE_TYPE,
            "path": DATA_PATH,
            "num_chunks": len(text_chunks) if text_chunks else 0,
            "chunk_size": CHUNK_SIZE,
            "chunk_overlap": CHUNK_OVERLAP,
        },
        "embedding": {
            "model": EMBEDDING_MODEL_NAME,
            "device": MODEL_KWARGS["device"],
        },
        "vectordb": {
            "path": VECTOR_DB_PATH,
            "retriever_k": RETRIEVER_K,
        },
        "llm": {
            "model": LLM_MODEL_NAME,
            "max_length": LLM_MAX_LENGTH,
            "temperature": LLM_TEMPERATURE,
            "top_p": LLM_TOP_P,
        },
        "rag_chain": {
            "chain_type": CHAIN_TYPE,
        }
    }

    config_path = f"{RESULTS_DIR}/config.json"
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)

    print(f"✓ Configuration saved to {config_path}")
    return config

# Save configuration
config = save_project_config()

print("\\n" + "=" * 70)
print("PROJECT SUMMARY")
print("=" * 70)
print(f"Project: {config['project_name']}")
print(f"Timestamp: {config['timestamp']}")
print(f"Results directory: {RESULTS_DIR}")
print(f"\\nKey Components:")
print(f"  - Data chunks: {config['data']['num_chunks']}")
print(f"  - Embedding model: {config['embedding']['model']}")
print(f"  - LLM: {config['llm']['model']}")
print(f"  - Retriever K: {config['vectordb']['retriever_k']}")

✓ Configuration saved to ./results/20251125_120712/config.json
\n======================================================================
PROJECT SUMMARY
Project: LMKR-RAG-Chatbot
Timestamp: 20251125_120712
Results directory: ./results/20251125_120712
\nKey Components:
  - Data chunks: 11
  - Embedding model: sentence-transformers/all-MiniLM-L6-v2
  - LLM: mistralai/Mistral-7B-Instruct-v0.1
  - Retriever K: 1


**What this cell does:**
- Saves all configurations for reproducibility
- Documents experiment settings
- Creates project summary

**Why It Matters:**
- Enables reproducible experiments
- Documents what settings worked best
- Useful for sharing with teammates
- Easy to restart/continue work